Hello! I am Floris.

https://www.florisvanvugt.com/

* Open for questions at any time. Love interruptions. What matters is that you can follow along.

* Let's take regular breaks. Body movement is important. If you feel you need to move, please tell me. That's not lazy, that's taking care of your needs.

In [ ]:
# The bureaucratics
%pylab inline
#import matplotlib.pyplot as plt
import numpy as np
import ipywidgets
from ipywidgets import interactive
from IPython.display import Audio, Image, Video, display, clear_output
from IPython.core.display import HTML 
import scipy.io.wavfile
import scipy.signal

Also you will need sound files (see Github repo)

# 1. Creating a simple sound wave

In [ ]:
Image(filename='music_everywhere.png')

Q: What is sound?

Q: What are properties of sounds?

* The simplest sounds are pure tones: air pressure is a sine wave function of time.
* The frequency of this sine wave determines the pitch of the sound (how high it sounds)

In [ ]:
# Creating a simple sine wave sound of a frequency of 440 Hz.
# I won't explain the details of this right now, just to get us started!
f = 440
SR = 44100
t = arange(0,1,1/SR)
signal = np.sin(2*pi*f*t)
display(Audio(data=signal, rate=SR))

To understand the above code, we need to understand something about how computers represent sounds.

* Sound waves are continuous. There is a particular air pressure at every moment in time which is just a little different from every other moment in time.
* However computers cannot deal with continuous signals that well. So instead we approximate the sound wave by a set of measurements at different points in time.
* We refer to these measurements as **samples**, and the rate by which they occur in time as the **sampling rate** (samples per time). Typically that's a very high number, like 44100 per second, i.e. 44.1 kHz.
* The higher the sampling rate, the more it will approximate the real, continuous sound wave. The better it will sound.

In [ ]:
Image(filename='reconstructing-the-original-signal.jpeg') ## source: https://www.izotope.com/en/learn

To determine at what times we will take a sample, we make a list of time points using `arange`.

In [ ]:
# Examples of using arange(from,to,step) (you have it in Matlab too I think)
arange(0,100,5)

In [ ]:
arange(0,1,.1)

In [ ]:
# Now back to our example:
SR = 100 # very low!
t = arange(0,1,1/SR)
t

In [ ]:
# Now back to our example:
SR = 44100 # more typical
t = arange(0,1,1/SR)
t

Sampling rate typically is expressed in Hz, which is number/second.

A sampling rate of 44100 means 44100 samples per second. We also often write 44.1 kHz.

Now that we have the time vector we can create the desired air pressure signal.

**Frequency** of a sound is the number of cycles per unit of time.

For example, the number of "waves" during 1 second. Expressed in Hz again.

There is a formula:
    
$$\textrm{signal}=\sin(2\pi*f*t)$$

Where $t$ is time, and $f$ is the frequency (in "cycles" per time).

We use the `numpy.sin()` function to calculate the sine wave.

In [ ]:
signal = np.sin(2*pi*f*t)
signal[:100]

Those are called **samples**, and the rate at which they occur is the **sampling rate** (taux d'échantillonnage).

In [ ]:
plot(signal[:100])

In [ ]:
SR = 44100
#SR = 2000
#SR = 950
#SR = 800
#SR = 700
f = 440
t = arange(0,1,1/SR)
signal = np.sin(2*pi*f*t)

plot(t,signal)
xlim(0,.005)

display(Audio(data=signal, rate=SR))

In [ ]:
## Advanced

SRs = [44100,2000,800,700]

f = 440

t_hires = arange(0,1,1/max(SRs))
s_hires = np.sin(2*pi*f*t_hires)

fig,axs = subplots(len(SRs),1,sharex=True,figsize=(8,8))
for ax,SR in zip(axs,SRs):
    t = arange(0,1,1/SR)
    signal = np.sin(2*pi*f*t)
    ax.plot(t_hires,s_hires,color='gray')
    ax.plot(t,signal,'o-',color='blue')
    ax.set_xlim(0,.005)
    ax.set_title("rate = %.0f Hz"%SR)
tight_layout()

### Summary so far

* Sound waves are variations in air pressure over time.
* The simplest sounds are pure tones, where air pressure is a sine wave function of time.
* Although sound waves are continuous, we measure them in discrete samples, at a particular sampling rate (samples per time)
* The higher the sampling rate, the more fidelity (being truthful to the original)

**Pitch** refers to the *perceptual* quality that some sounds are higher and some lower. This perceptual quality corresponds to the physical property of frequency.

In [ ]:
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/qNf9nzvnd1k?rel=0&amp;controls=1&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

## Output to WAVE file

scipy.io.wavfile.write(filename, rate, data)

Don't you just love the internet? https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.wavfile.write.html

In [ ]:
scipy.io.wavfile.write('a440.wav',SR,signal)

In [ ]:
rt,dt = scipy.io.wavfile.read('a440.wav')

In [ ]:
rt

In [ ]:
dt

In [ ]:
plot(dt[:100])

## Amplitude

What makes some sounds louder than others?

In [ ]:
def play_nonorm_unit(data,rate):
    # I'll tell you later what these two commands are doing here:
    dat = array(data) # ensure editable
    dat[-1]=1    # disable normalization for signal [-1,1]
    display(Audio(data=dat, rate=rate))

In [ ]:
f = 440
t = arange(0,1,1/SR)
signal = .5 * np.sin(2*pi*f*t) # modify amplitude
play_nonorm_unit(signal,SR)

**Amplitude** can be thought of as the difference between the maximum and the minimum value of the signal.

The greater the amplitude, the greater the changes in air pressure, and the louder the sound. 

### Summary so far

* The **frequency** of a sound corresponds to its perceived pitch.
* The **amplitude** of a sound corresponds to its perceived loudness.

# 2. Basic sound editing issues

## 2.1 Fading

Did this click at the beginning and end annoy you too?

In [ ]:
f = 342.1
SR = 44100
t = linspace(0,1,SR)
signal = np.sin(2*pi*f*t)
display(Audio(data=signal, rate=SR))

In [ ]:
plot(signal[:300])

What a cliffhanger!!

Q: What can we do about it?

Fading! That is, changing the amplitude of the sound gradually from zero to full at the beginning, and then back from full to zero at the end.

In [ ]:
plot(signal)

Let's fade in the entire sound first (easier).

In [ ]:
fader = linspace(0,1,SR)

Intermezzo: `linspace(from,to,n)` (à ne pas confondre avec `arange()`)

In [ ]:
linspace(0,10,3)

In [ ]:
linspace(0,10,21)

In [ ]:
plot(fader)

In [ ]:
newsignal = fader*signal
display(Audio(data=newsignal, rate=SR))

In [ ]:
plot(newsignal)

**EXERCISE** Can you instead fade out the sound?

Now let's say we want to fade in, then plateau, and then fade out.

In [ ]:
n_samples_fade = int(.1*SR)
n_samples_plateau = int(.8*SR)

In [ ]:
fade_in  = linspace(0,1,n_samples_fade)
plateau  = ones(n_samples_plateau)
fade_out = linspace(1,0,n_samples_fade)

envelope = concatenate([fade_in,plateau,fade_out])

In [ ]:
plot(envelope)

In [ ]:
newsignal = envelope*signal
display(Audio(data=newsignal, rate=SR))

In [ ]:
plot(newsignal)

**EXERCISE** Can you make the fade take longer, i.e. 200ms instead of 100ms?

## 2.2 Volume normalization

Sound source: freesound.org

In [ ]:
def play_nonorm(data,rate):
    # I'll tell you later what these two commands are doing here:
    dat = array(data) # ensure editable
    dat[-1]=32767    # disable normalization
    display(Audio(data=dat, rate=rate))

In [ ]:
rt,sneeze = scipy.io.wavfile.read('sneeze.wav') 
## reading a wave sound gives you two variables, the first is the sampling rate,
## and the second is the actual sound data.
play_nonorm(sneeze,rt)

In [ ]:
rt,whisper = scipy.io.wavfile.read('whisper.wav')
play_nonorm(whisper,rt)

In [ ]:
plot(sneeze)

In [ ]:
sneeze.shape # that many samples

**EXERCISE** Can you create a time vector to go with the signal? You know the signal sampling rate, you know how many samples...

In [ ]:
n = sneeze.shape[0]
t = arange(n)/rt # create a corresponding time vector
# arange(n) is short for arange(0,n,1)
plot(t,sneeze)

Ok, let's plot them together. What do you expect to see?

In [ ]:
t = arange(0,sneeze.shape[0])/rt # create a corresponding time vector
plot(t,sneeze)
t2 = arange(0,whisper.shape[0])/rt # create a corresponding time vector
plot(t2,whisper)

Q: How can we make the volume more similar?

In [ ]:
play_nonorm(5*whisper,rt)

Ok, but can we be a bit more systematic about it?

### 2.2.1 Peak normalization

One way: take the maximum and make it the same across both sounds.

**EXERCISE** Can you rescale the amplitude of the sneeze so that its maximum is 30000? (yes that's an arbitrary number)

In [ ]:
sneezenorm = 30000*(sneeze/max(sneeze))

In [ ]:
plot(sneezenorm)

In [ ]:
play_nonorm(sneezenorm,rt)

In [ ]:
whispernorm = 30000*(whisper/max(whisper))

In [ ]:
play_nonorm(whispernorm,rt)

Q: What is the limitation of using peak normalization?

### 2.2.2 RMS normalization

The idea: take the square, then take the mean, then take the root of the signal. You should get one value.

RMS square -> mean -> root

mathematically: root(mean(square(x)))

Toy example: [2,1,0,1,2] 
What is the RMS?

squares: 4,1,0,1,4, mean = 10/5=2, sqrt = sqrt(2)~1.41

In [ ]:
x = array([2,1,0,1,2])

In [ ]:
sqrt(mean(x**2))

Now with real sounds...

In [ ]:
sneeze = array(sneeze,dtype=float32) # don't worry about why this is necessary

In [ ]:
sneeze**2

In [ ]:
power(sneeze,2)

In [ ]:
mean(power(sneeze,2))

In [ ]:
sqrt(mean(power(sneeze,2)))

In [ ]:
whisper = array(whisper,dtype=float32) # don't worry about why this is necessary

In [ ]:
sqrt(mean(power(whisper,2))) # what do you expect about the value that you'll get?

In [ ]:
sneeze_rms = sneeze/ sqrt(mean(power(sneeze,2)))

In [ ]:
whisper_rms = whisper/ sqrt(mean(power(whisper,2)))

In [ ]:
plot(sneeze_rms)
plot(whisper_rms)

In [ ]:
play_nonorm(whisper_rms,rt)

Q: WTF?

Ah right, we normalized both sounds to a really really weak volume!

In [ ]:
play_nonorm(30000*whisper_rms,rt)

In [ ]:
play_nonorm(30000*sneeze_rms,rt)

# 3 Sounds coming together

In [ ]:
# play_nonorm(15000*(whisper_rms+sneeze_rms),rt)

## 3.1 Musical chords

In [ ]:
SR = 44100
f1 = 440
f2 = 660
t = arange(0,1,1/SR)
signal = np.sin(2*pi*f1*t)+np.sin(2*pi*f2*t)
display(Audio(data=signal, rate=SR))

In [ ]:
plot(t,signal); xlim(0,.05)

In [ ]:
duration = 3
def two_frequencies(f1=220.0, f2=224.0):
    t = arange(0,duration,1/SR)
    signal = np.sin(2*np.pi*f1*t) + np.sin(2*np.pi*f2*t)
    display(Audio(data=signal, rate=SR))
    return signal

v = interactive(two_frequencies, f1=(200.0,300.0), f2=(200.0,300.0))
display(v)

## 3.2 Musical sequences

What if we want to present sounds one after another?

In [ ]:
f1 = 440
f2 = 660
t = arange(0,1,1/SR)
sound1 = np.sin(2*pi*f1*t)
sound2 = np.sin(2*pi*f2*t)
signal = concatenate([sound1,sound2])
display(Audio(data=signal, rate=SR))

This can also be done in a loop

In [ ]:
sounds = []
t = arange(0,1,1/SR)
for f in [440,550,660,880,660,550,440]:
    sound = np.sin(2*pi*f*t)
    sounds.append(sound)

In [ ]:
sounds

In [ ]:
signal = concatenate(sounds)
display(Audio(data=signal, rate=SR))

**EXERCISE**: can you allow as part of the loop to have sounds of different durations as well?

In [ ]:
sounds = []
for f,dur,sil in [
    (264,.250,.5),
    (264,.250,.25),
    (297,1,.25),
    (264,1,.25),
    (352,1,.25),
    (330,2,.5)]:
    t = arange(0,dur,1/SR)
    sound = np.sin(2*pi*f*t)
    sounds.append(sound)
    sounds.append(zeros(int(sil*SR)))
signal = concatenate(sounds)
display(Audio(data=signal, rate=SR))

## 3.3 Forever rising...

Can we have sounds that go on rising?

First step: create all octaves (= doubling of frequencies)

In [ ]:
t = arange(0,1,1/SR)
sounds = zeros(1*SR)
for f in [110, 220, 440, 880, 1760, 3520, 7040, 14080, 28160, 56320, 112640]:
    sound = np.sin(2*pi*f*t)
    sounds += sound

In [ ]:
display(Audio(data=sounds, rate=SR))

Typical hearing of humans is from 20 Hz to 20 kHz.

In [ ]:
f = 20
f*(2**arange(11)) # roughly the frequencies we need

In [ ]:
basef = 20
t = linspace(0,1,SR)
sounds = zeros(1*SR)
for f in basef*(2**arange(11)):
    sound = np.sin(2*pi*f*t)
    sounds += sound
display(Audio(data=sounds, rate=SR))

**EXERCISE** Wrap that in a function that takes base frequency as argument.

In [ ]:
def alloctaves(basef,dur=1):
    # Create a sound that contains lots of octaves above and below
    # a certain base frequency (basef) and that lasts dur seconds
    t = arange(0,dur,1/SR)
    sounds = zeros(int(dur*SR))
    for f in basef*(2**arange(11)):
        sound = np.sin(2*pi*f*t)
        sounds += sound
    return sounds

In [ ]:
display(Audio(data=alloctaves(23), rate=SR)) 

In [ ]:
def dispoct(f):
    display(Audio(data=alloctaves(f), rate=SR)) 
v = interactive(dispoct, f=(20,40))
display(v)

**EXERCISE** Can you make a sequence of three shepard tones with base frequencies 20, 21 and 22 Hz?

In [ ]:
sounds = []
for f in range(20,40,1):
    sound = alloctaves(f,.25)
    sounds.append(sound)
    sounds.append(zeros(int(.05*SR)))
signal = concatenate(sounds)
signal = concatenate([signal,signal,signal,signal])
display(Audio(data=signal, rate=SR))

Welcome to Shepard tones! https://en.wikipedia.org/wiki/Shepard_tone

# 4. Frequency analysis

In [ ]:
sound = alloctaves(20,1)

In [ ]:
plot(sound)

How can we find what frequencies were part of this sound?

That's where Fourier analysis comes in.

Fourier Transform allows us to write any signal as a combination of pure sine waves. So it's doing the inverse of what we've been doing so far, namely combining pure sound waves into more complex sounds.

The human ear also does something similar.

In [ ]:
Video("2---Cochlear animation.mp4")

In [ ]:
Image(url= "https://4d6619e6-a-62cb3a1a-s-sites.googlegroups.com/site/jayanthinyswebsite/workshops/different%20aspects%20of%20organ%20of%20Corti.png")

There is a bunch of detail in terms of math that is not too important here. What is important is the idea that you have a signal, and then it splits it into the various frequencies.

In [ ]:
fourierTransform = np.fft.fft(sound)/len(sound)           # Normalize amplitude
fourierTransform = fourierTransform[range(int(len(sound)/2))] # Exclude sampling frequency
tpCount     = len(sound)
values      = np.arange(int(tpCount/2))
timePeriod  = tpCount/SR
frequencies = values/timePeriod

In [ ]:
plot(frequencies, abs(fourierTransform))
xlim(0,200)

We can get the amplitude too!

In [ ]:
f1 = 440
f2 = 660
t = arange(0,1,1/SR)
sound = np.sin(2*pi*f1*t)+.5*np.sin(2*pi*f2*t)

In [ ]:
fourierTransform = np.fft.fft(sound)/len(sound)           # Normalize amplitude
fourierTransform = fourierTransform[range(int(len(sound)/2))] # Exclude sampling frequency
tpCount     = len(sound)
values      = np.arange(int(tpCount/2))
timePeriod  = tpCount/SR
frequencies = values/timePeriod
plot(frequencies, abs(fourierTransform))
xlim(400,800)

In [ ]:
Image(url= "https://www.nti-audio.com/portals/0/pic/news/FFT-Time-Frequency-View-540.png")

In [ ]:
sounds = []
for f in [440,550,660,880,660,550,440]:
    sound = np.sin(2*pi*f*t)
    sounds.append(sound)
data = concatenate(sounds)
#signal = concatenate([signal])
display(Audio(data=data, rate=SR))

specgram(data,Fs=SR) 
ylabel('Frequency [Hz]')
xlabel('Time [sec]')
ylim(0,2500)

In [ ]:
rt,data = scipy.io.wavfile.read('youare.wav') # source https://www.101soundboards.com
display(Audio(data=data, rate=rt))
specgram(data,Fs=rt) 
ylabel('Frequency [Hz]')
xlabel('Time [sec]')
ylim(0,15000)

# 5. Simple experiment: mapping positions to sounds

van Vugt & Ostry "audiomotor" paradigm.

http://www.psych.mcgill.ca/labs/mcl/pdf/jocn_a_01204.pdf

In [ ]:
Image(url="https://pbs.twimg.com/media/EHFzGNGXkAEzcv2.jpg")

In [ ]:
# A simplified mapping between position and sound:
def angle2sound(angle):
    # Angle in degrees, between 0 and 180 degrees
    f1 = 400+2*angle
    f2 = 1600-2*angle
    sound = np.sin(2*pi*f1*t)+np.sin(2*pi*f2*t)
    return sound

In [ ]:
display(Audio(data=angle2sound(0),rate=SR))

## Creating an interface

Buttons, sliders

In [ ]:
button_next = ipywidgets.Button(description="Choose next target")
response = ipywidgets.IntSlider(value=90,min=0,max=180,step=1,
                                description='What angle do you think corresponds to that sound?')
button_respond = ipywidgets.Button(description="Select response")
display(button_next,response,button_respond)

In [ ]:
record = {}
record["target_angle"] = 180*random.random()

def select_target(e):
    snd = angle2sound(record["target_angle"])
    clear_output()
    display(ipywidgets.Label(value="This is your target sound:"))
    display(Audio(data=snd,rate=SR))
    # add buttons for responding
    display(response)
    display(button_respond)

In [ ]:
button_next.on_click(select_target)
display(button_next)

In [ ]:
def register_response(e):
    val = response.value
    # Give the feedback sound
    snd = angle2sound(val)
    clear_output()
    display(ipywidgets.Label(value="This is your feedback sound for an angle of "+str(val)))
    display(Audio(data=snd,rate=SR))
    display(button_next)

In [ ]:
button_respond.on_click(register_response)
display(button_next)

But so far we don't keep a record of the subject's performance..

In [ ]:
record = {'trials':[]}
record["target_angle"] = 180*random.random()

def register_response(e):
    val = response.value
    record["trials"].append({'target_angle':record["target_angle"],
                            'response_angle':val})
    # Give the feedback sound
    snd = angle2sound(val)
    clear_output()
    display(ipywidgets.Label(value="This is your feedback sound for an angle of "+str(val)))
    display(Audio(data=snd,rate=SR))
    display(button_next)
    
button_respond.on_click(register_response)
display(button_next)

In [ ]:
record["trials"]

In [ ]:
trials = record['trials']
for trial in trials:
    trial['error']=trial['target_angle']-trial['response_angle']

In [ ]:
errors = [ trial['error'] for trial in trials ]
plot(errors)

# That's all folks